# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 32.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city politically and historically, serving as the hub for the federal government's three branches. If you have any more questions about it or need further clarification, feel free to ask!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  98


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss various aspects of generative AI, including its applications, impact, and controversies. Some tweets highlight new AI models and tools, such as Microsoft's free course on generative AI and RunwayML's Gen-4 models. Others express concerns about the environmental impact and ethical implications of generative AI, particularly in art and creative industries. There are mentions of collaborations, like NVIDIA and AWS working to make AI more accessible, and discussions on the future trends of AI, including its potential in healthcare and media. Additionally, some tweets criticize the use of AI in creative fields, advocating for human artists, while others express excitement about AI's potential to revolutionize content creation.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

Generative AI is rapidly advancing, impacting industries from art to healthcare, with both excitement and criticism surrounding its use.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

People discuss AI with a mix of excitement and skepticism. Generative AI is seen as transformative for media and business, yet criticized for its environmental impact and lack of originality. Some celebrate its potential, while others advocate for traditional artistry and caution against its unchecked use.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [17]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
RT @Artistreccs: Made this account as a huge fuck you to Ai generative art and the the rest of them losers tbh 

Commission real artists 👍
===
harassment
RT @imzeferino: generative AI art is is soulless, sad and pathetic... i won't even bother calling it ugly, because that is beside the point…
===
harassment
RT @Artistreccs: Made this account as a huge fuck you to Ai generative art and the the rest of them losers tbh 

Commission real artists 👍
===
harassment
RT @Artistreccs: Made this account as a huge fuck you to Ai generative art and the the rest of them losers tbh 

Commission real artists 👍
===
harassment
RT @Artistreccs: Made this account as a huge fuck you to Ai generative art and the the rest of them losers tbh 

Commission real artists 👍
===
violence
The IDF, one of the leaders in the tech industry, uses and trains generative ai off the killing of real human beings. This technology would give a government an army without needing human soldiers


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [18]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

RT @16pxl: 只是发布我这个堕落的人类制作的吉卜力风格像素艺术，而不是生成式AI 🤪 没有任何理由！https://t.co/b…
RT @MoureDev: 微软发布了其官方课程的新版本，用于学习生成式人工智能。

这是免费的，并通过21节课教你…
RT @runwayml: 今天我们推出了Gen-4，我们的新一代尖端AI模型系列，用于媒体生成和世界一致性。Gen-4…
软件和互联网公司在未来三年内，来自生成式人工智能的收入可能会增长超过20倍，并且最早在2025年就能实现正投资回报。https://t.co/ztP34WsIRC
@poiyomi @HOUNDDS 许多平台都实施了生成式AI，但没有提供不使用它的选项。而且，大多数时候它甚至不准确。进行简单的谷歌搜索不应该使用生成式AI。以前在谷歌上进行研究效果很好。
@sawyomom 是的，因为他是真正的人工智能。一个能够独立思考的完整大脑 ≠ 生成脚本
RT @runwayml: 今天我们推出了Gen-4，我们的新一代尖端AI模型系列，用于媒体生成和世界一致性。Gen-4…
RT @Artistreccs: 创建这个账号是对AI生成艺术和其他那些失败者的一个巨大反击，说实话

委托真正的艺术家 👍
RT @jblefevre60: 💥8个生成式AI流行词！

#AI #机器学习 #深度学习 #数据科学 #生成式AI #LLM #Python #编程 #100天…
Theta 的多操作系统边缘节点支持是一个改变游戏规则的创新——就像给每个设备提供了进入去中心化计算的后台通行证！与 AWS 和三星合作，它正在推动从生成式 AI 到元链交易的创新。*喘气*
@Writing_Dragons 我认为使用支持AI的软件与能够充当人类编辑的生成式AI有很大不同。我认为区别在于你作为作者是否仍然掌控局面。
热烈祝贺@MahojinAI，这是一个基于Story构建的生成式AI混音工具 ↴ https://t.co/7L7DLVeoeU
RT @AdrianEarthmeta: 🧠 你对下一个人工智能大趋势有什么预测？
🤖 超逼真的AI聊天机器人？🧬 突破性…
RT @freezetheberry: 在生成式人工智能“艺术”日益流行的时代，我们有这样的艺术家证明机器永远无法替代……
RT @jblefevre60: 

In [19]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

RT @16pxl: Ah, behold my Ghibli-inspired pixel art, crafted by a mere mortal rather than some soulless generative AI. No particular reason, just felt like it! 🤪 https://t.co/b…
Ah, splendid! Microsoft has unveiled a fresh iteration of their official course on Generative AI. And guess what, it's absolutely free! In just 21 lessons, you'll be on your way to mastering the art of artificial intelligence. How delightfully efficient!
RT @runwayml: Ah, splendid news, isn't it? Today we unveil Gen-4, our latest series of state-of-the-art AI models, designed for media generation and world consistency. Gen-4, the pièce de résistance of technological advancement...
Oh, splendid! It appears those software and internet companies are about to experience a delightful surge in revenue from Generative AI, skyrocketing over 20 times in the next three years. And by 2025, they'll be swimming in positive returns on investment. How positively thrilling! https://t.co/ztP34WsIRC
Oh, how utterly delightful! It

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [20]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "playful",
  "mentioned": ["@16pxl"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["MoureDev", "Microsoft"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["runwayml"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "optimism",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["@poiyomi", "@HOUNDDS"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "admiration",
  "mentioned": ["sawyomom"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "anticipation",
  "mentioned": ["runwayml"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "anger",
  "mentioned": ["Artistreccs"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": [
    "jblefevre60"
  ],
  "support": "neut

KeyboardInterrupt: 

In [21]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Generative AI in Art",
    "Generative AI in Media and Content Creation",
    "Generative AI in Education and Courses",
    "Generative AI in Business and Industry",
    "Generative AI and Environmental Concerns",
    "Generative AI in Gaming and Entertainment",
    "Generative AI in Healthcare and Medical Research",
    "Generative AI and Ethical Concerns",
    "Generative AI in Marketing and Advertising",
    "Generative AI and Technological Advancements"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [22]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 98/98 [01:30<00:00,  1.09it/s]


In [23]:
print(analysis_result)

['{\n  "sentiment": "neutral",\n  "emotion": "playful",\n  "mentioned": ["16pxl"],\n  "support": ""\n}', '{\n  "sentiment": "positive",\n  "emotion": "informative",\n  "mentioned": ["MoureDev", "Microsoft"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["runwayml"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "optimism",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["@poiyomi", "@HOUNDDS"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "admiration",\n  "mentioned": ["sawyomom"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["runwayml"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "anger",\n  "mentioned": [\n    "Artistreccs"\n  ],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "in

In [24]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 0,
  "Republican count": 0,
  "people name": {
    "16pxl": {
      "sentiments": ["neutral"],
      "emotions": ["playful"]
    },
    "MoureDev": {
      "sentiments": ["positive"],
      "emotions": ["informative"]
    },
    "Microsoft": {
      "sentiments": ["positive"],
      "emotions": ["informative"]
    },
    "runwayml": {
      "sentiments": ["neutral"],
      "emotions": ["informative", "anticipation"]
    },
    "@poiyomi": {
      "sentiments": ["negative"],
      "emotions": ["frustration"]
    },
    "@HOUNDDS": {
      "sentiments": ["negative"],
      "emotions": ["frustration"]
    },
    "sawyomom": {
      "sentiments": ["neutral"],
      "emotions": ["admiration"]
    },
    "Artistreccs": {
      "sentiments": ["negative"],
      "emotions": ["anger"]
    },
    "jblefevre60": {
      "sentiments": ["neutral"],
      "emotions": ["informative"]
    },
    "AWS": {
      "sentiments": ["positive"],
      "emotions": ["excitement"]
    },


## Create a chatbot

In [25]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  what do they talk about?


Chatbot: The tweets cover a range of topics related to generative AI:

1. **Art and Creativity:**
   - There are discussions about the role of generative AI in art, with some users criticizing it for being less authentic and others defending traditional art by human artists.

2. **Educational Resources:**
   - Mention of a free course by Microsoft aimed at teaching people about generative AI.

3. **Technological Advancements:**
   - New AI models such as Gen-4 by Runway ML are introduced, showcasing advancements in media generation and world consistency.

4. **Industry Collaboration:**
   - Companies like NVIDIA, AWS, and Samsung are teaming up to enhance the accessibility and impact of generative AI.

5. **Environmental and Ethical Concerns:**
   - Criticisms about the environmental impact of generative AI due to high energy consumption and ethical concerns regarding its use in creative fields.

6. **Speculations on Future Trends:**
   - Predictions about future trends in AI, includin

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
